In [1]:
import os
os.getpid()

2023

In [3]:
import sys
sys.executable

'/home/dev2/anaconda3/envs/abzooba/bin/python3.9'

In [10]:
from pathlib import Path

import pandas as pd
import numpy as np

from supervised.automl import AutoML

In [12]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [14]:
train_df.columns

Index(['id', 'county', 'city', 'zipcode', 'address', 'state', 'latitude',
       'longitude', 'cemetery_dist_miles', 'nationalhighway_miles',
       'railline_miles', 'starbucks_miles', 'walmart_miles', 'hospital_miles',
       'physician_dist_miles', 'dentist_dist_miles', 'opt_dist_miles',
       'vet_dist_miles', 'farmers_miles', 'bed', 'bath', 'halfbath', 'sqft',
       'property_type', 'garage', 'yearbuilt', 'pool', 'fireplace', 'patio',
       'lotsize', 'Census_MedianIncome', 'Unemployment', 'EmploymentDiversity',
       'Crime_Rate', 'rent'],
      dtype='object')

In [33]:
train_df.sample(2)

,id,county,city,zipcode,address,state,latitude,longitude,cemetery_dist_miles,nationalhighway_miles,railline_miles,starbucks_miles,walmart_miles,hospital_miles,physician_dist_miles,dentist_dist_miles,opt_dist_miles,vet_dist_miles,farmers_miles,bed,bath,halfbath,sqft,property_type,garage,yearbuilt,pool,fireplace,patio,lotsize,Census_MedianIncome,Unemployment,EmploymentDiversity,Crime_Rate,rent
15412,train_15412,PHILADELPHIA,PHILADELPHIA,19146.0,2640 ELLSWORTH ST,PA,39.939337,-75.187899,0.944864,0.040771,0.116956,0.975726,2.585405,0.979081,0.287587,0.705438,0.881885,0.972581,0.260486,1,1,1,925,Condo,0,2016,0,0,0,1064.0,21561.50,6.0,3.46,6.372162,1450
21005,train_21005,DAUPHIN,MIDDLETOWN,17057.0,1885 LAKESIDE DR,PA,40.207958,-76.775521,2.293343,0.297274,0.376008,0.934528,3.650832,6.227922,1.102863,1.294007,1.746795,54.666808,3.057894,3,1,1,1200,Condo,0,1960,0,0,0,2400.0,52986.87,4.5,3.57,1.800000,1045


In [35]:
train_df['county'].value_counts()

PHILADELPHIA    6674
ALLEGHENY       3460
MONTGOMERY      1922
DELAWARE        1210
CHESTER         1143
                ... 
VENANGO            2
JEFFERSON          2
SULLIVAN           1
ARMSTRONG          1
JUNIATA            1
Name: county, Length: 64, dtype: int64

In [38]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24269 entries, 0 to 24268
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     24269 non-null  object 
 1   county                 24235 non-null  object 
 2   city                   24269 non-null  object 
 3   zipcode                24235 non-null  float64
 4   address                23317 non-null  object 
 5   state                  24269 non-null  object 
 6   latitude               24269 non-null  float64
 7   longitude              24269 non-null  float64
 8   cemetery_dist_miles    24269 non-null  float64
 9   nationalhighway_miles  24269 non-null  float64
 10  railline_miles         24269 non-null  float64
 11  starbucks_miles        24269 non-null  float64
 12  walmart_miles          24269 non-null  float64
 13  hospital_miles         24269 non-null  float64
 14  physician_dist_miles   24269 non-null  float64
 15  de

In [17]:
train_df.describe()

,zipcode,latitude,longitude,cemetery_dist_miles,nationalhighway_miles,railline_miles,starbucks_miles,walmart_miles,hospital_miles,physician_dist_miles,dentist_dist_miles,opt_dist_miles,vet_dist_miles,farmers_miles,bed,bath,halfbath,sqft,garage,yearbuilt,pool,fireplace,patio,lotsize,Census_MedianIncome,Unemployment,EmploymentDiversity,Crime_Rate,rent
count,24235.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000,24269.000000
mean,18030.844564,40.080788,-77.241598,1.206987,0.290908,1.049048,2.042534,3.528326,2.269302,0.352973,0.712536,0.892396,12.419132,1.769471,2.766863,1.621122,1.026989,1535.360625,0.291895,1958.995014,0.011455,0.206972,0.197948,3669.835108,57016.945681,5.187826,3.438248,6.394034,1599.895505
std,1477.571555,1.637096,5.817313,0.927508,0.364962,1.574878,3.205261,1.794201,2.467797,0.551741,0.993609,1.092290,17.585042,2.204656,1.446065,0.854438,0.212449,824.447982,0.454644,23.278187,0.106415,0.405144,0.398461,3185.150540,23918.805643,0.956070,0.150563,7.345398,916.256058
min,15001.000000,-12.116930,-149.900278,0.022214,0.000027,0.000248,0.013197,0.118545,0.041096,0.000087,0.000157,0.000157,0.013511,0.012470,1.000000,1.000000,1.000000,35.000000,0.000000,1800.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.970000,2.350000,0.000000,385.000000
25%,17111.000000,39.966766,-77.008588,0.818566,0.143624,0.465909,0.959076,2.712467,1.160151,0.170974,0.359750,0.490856,4.794041,0.740475,2.000000,1.000000,1.000000,1000.000000,0.000000,1960.000000,0.000000,0.000000,0.000000,1712.000000,40817.450000,4.500000,3.370000,1.468825,1000.000000
50%,18944.000000,40.130169,-75.475763,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,3.000000,1.000000,1.000000,1343.000000,0.000000,1960.000000,0.000000,0.000000,0.000000,2872.730796,54476.090000,5.100000,3.480000,2.400000,1400.000000
75%,19128.000000,40.444389,-75.185042,1.227709,0.281964,0.860038,1.789913,3.870699,2.196950,0.299176,0.611134,0.853274,10.299977,1.703883,3.000000,2.000000,1.000000,1839.000000,1.000000,1960.000000,0.000000,0.000000,0.000000,4266.189119,70379.790000,6.000000,3.540000,9.136104,1895.000000
max,19611.000000,61.218056,-11.706829,9.747778,7.920969,28.306430,63.618152,26.409748,32.433629,14.301216,19.823016,22.761643,141.184122,35.603985,18.000000,12.000000,12.000000,5000.000000,1.000000,2016.000000,1.000000,1.000000,1.000000,19341.000000,198425.570000,10.000000,3.670000,39.159639,16000.000000


In [42]:
train_df.isnull().sum()

id                          0
county                     34
city                        0
zipcode                    34
address                   952
state                       0
latitude                    0
longitude                   0
cemetery_dist_miles         0
nationalhighway_miles       0
railline_miles              0
starbucks_miles             0
walmart_miles               0
hospital_miles              0
physician_dist_miles        0
dentist_dist_miles          0
opt_dist_miles              0
vet_dist_miles              0
farmers_miles               0
bed                         0
bath                        0
halfbath                    0
sqft                        0
property_type            3626
garage                      0
yearbuilt                   0
pool                        0
fireplace                   0
patio                       0
lotsize                     0
Census_MedianIncome         0
Unemployment                0
EmploymentDiversity         0
Crime_Rate

In [44]:
train_df[train_df['county'].isnull()]

,id,county,city,zipcode,address,state,latitude,longitude,cemetery_dist_miles,nationalhighway_miles,railline_miles,starbucks_miles,walmart_miles,hospital_miles,physician_dist_miles,dentist_dist_miles,opt_dist_miles,vet_dist_miles,farmers_miles,bed,bath,halfbath,sqft,property_type,garage,yearbuilt,pool,fireplace,patio,lotsize,Census_MedianIncome,Unemployment,EmploymentDiversity,Crime_Rate,rent
0,train_0,NaN,WEXFORD,NaN,266 Clematis Dr Allegheny County,PA,40.618200,-80.077600,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,3,2,1,2000,Condo,1,2008,0,1,0,4086.388045,54476.09,5.1,3.48,2.4,2400
1,train_1,NaN,WHITEHALL,NaN,2310 N 1st Ave Lehigh County,PA,40.649906,-75.478940,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,2,1,1,1100,Condo,0,1935,0,0,0,2247.513425,54476.09,5.1,3.48,2.4,995
2,train_2,NaN,WHITEHALL,NaN,3338 St Stephens Ln Lehigh County,PA,40.646282,-75.510056,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,3,2,1,1522,Condo,0,2006,0,1,1,3109.741302,54476.09,5.1,3.48,2.4,1740
3,train_3,NaN,WAYNESBORO,NaN,97 W Main St Franklin County,PA,39.756992,-77.579704,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,3,1,1,1150,Condo,0,1960,0,0,0,2349.673126,54476.09,5.1,3.48,2.4,675
4,train_4,NaN,QUAKERTOWN,NaN,200 E Broad St Bucks County,PA,40.441177,-75.332540,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,3,2,1,1000,SFR,0,1960,0,0,0,2043.194023,54476.09,5.1,3.48,2.4,1300
5,train_5,NaN,WAYNESBORO,NaN,407 Viewpoint Way Franklin County,PA,39.766594,-77.582490,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,3,3,1,1290,Condo,1,2012,0,0,0,2635.720289,54476.09,5.1,3.48,2.4,1025
6,train_6,NaN,WAYNESBORO,NaN,403 Viewpoint Way Franklin County,PA,39.766580,-77.582450,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,3,3,1,1290,Condo,1,2012,0,0,0,2635.720289,54476.09,5.1,3.48,2.4,1025
7,train_7,NaN,WAYNESBORO,NaN,240 Crown Ct Franklin County,PA,39.759663,-77.575090,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,3,3,1,1290,SFR,1,2007,0,0,0,2635.720289,54476.09,5.1,3.48,2.4,1025
8,train_8,NaN,WAYNESBORO,NaN,101 S Potomac St Franklin County,PA,39.755530,-77.581154,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,1,1,1,1200,Condo,1,1960,0,0,0,2451.832827,54476.09,5.1,3.48,2.4,795
9,train_9,NaN,UPPER DARBY,NaN,446 Woodcliffe Rd Delaware County,PA,39.958010,-75.264300,1.019586,0.206222,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,3,1,1,1004,Condo,0,1960,0,0,0,2051.366799,54476.09,5.1,3.48,2.4,925


In [21]:
train_spl_x = train_df.drop('rent', axis=1)
train_spl_y = train_df['rent']

In [51]:
automl = AutoML(mode='Explain', explain_level=2)

In [52]:
automl.fit(train_spl_x, train_spl_y,)

Linear algorithm was disabled.
AutoML directory: AutoML_1
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models


2021-09-24 18:29:43,301 supervised.preprocessing.eda ERROR There was an issue when running EDA. sequence item 29: expected str instance, float found


AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_Baseline rmse 919.185214 trained in 1.48 seconds
2_DecisionTree rmse 633.760158 trained in 11.75 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost rmse 352.822738 trained in 92.63 seconds
4_Default_NeuralNetwork rmse 514.448938 trained in 8.27 seconds
5_Default_RandomForest rmse 598.662216 trained in 12.65 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 352.822738 trained in 0.19 seconds
AutoML fit time: 132.91 seconds
AutoML best model: 3_Default_Xgboost


AutoML(explain_level=2)